In [354]:
import pandas as pd
import psycopg2 as pc
import mysql.connector as mc
import os
from dotenv import load_dotenv
import gspread
import gspread_dataframe as gd
import time
import datetime
from datetime import datetime, date
import numpy as np
import decimal

# KHAI BÁO GG SHEET ĐỔ DATA RAW

In [355]:
mail = gspread.service_account('credentials.json')
open = mail.open_by_url('https://docs.google.com/spreadsheets/d/1kJwGlIl5SEqFpV36cQFYXXZHkTNhHFA9KbjUKnX4RFg/edit#gid=0')
sheet1 = open.worksheet('DATA RAW')

# Đặt điều kiện

In [356]:
default_code = ('LO23301110',)

# Kết nối DB odoo mới, cũ và weblime

In [357]:
# Kết nối DB odoo mới
load_dotenv('.envnewodoo')
new_erp = pc.connect(
    host = os.getenv('new_host'),
    port = os.getenv('new_port'),
    database = os.getenv('new_db'),
    user = os.getenv('new_user'),
    password = os.getenv('new_password')
)

# Kết nối DB odoo cũ
load_dotenv('.envodoo')
old_erp = pc.connect(
    host = os.getenv('old_host'),
    port = os.getenv('old_port'),
    database = os.getenv('old_db'),
    user = os.getenv('old_user'),
    password = os.getenv('old_password')
)

# Kết nối DB weblime
load_dotenv('.envweb')
web = mc.connect(
    host = os.getenv('web_host'),
    port = os.getenv('web_port'),
    database = os.getenv('web_db'),
    user = os.getenv('web_user'),
    password = os.getenv('web_password')
)

# Query data Odoo cũ kết hợp với Category và collection của Odoo mới

In [358]:
old_cursor = old_erp.cursor()
old_cursor.execute(
    """
    SELECT
        pt.name AS pfcode,
        SPLIT_PART(pt.name,'-',1) AS default_code,
        SPLIT_PART(pt.name,'-',2) AS color,
        SPLIT_PART(pt.name,'-',3) AS size,
        s.nbdb,
        import.qty_import,
        SUM(pol.qty) AS qty_fm,
        SUM(pol.price_subtotal_incl) AS total_amount_fm
    FROM pos_order po
        LEFT JOIN pos_order_line pol ON po.id=pol.order_id
        LEFT JOIN product_product pp ON pol.product_id=pp.id
        LEFT JOIN product_template pt ON pp.product_tmpl_id=pt.id
        LEFT JOIN 
            (SELECT
                pol.product_id,
                MIN(DATE(po.date_order + '7 hour')) AS nbdb
            FROM pos_order po
                LEFT JOIN pos_order_line pol ON po.id=pol.order_id
            GROUP BY pol.product_id) AS s ON s.product_id=pp.id
         LEFT JOIN
         	(SELECT
					pp.id,
					import.qty_import - (CASE WHEN export.qty_export IS NULL THEN 0 ELSE export.qty_export END) AS qty_import
				FROM product_product pp
					LEFT JOIN product_template pt ON pp.product_tmpl_id=pt.id
					LEFT JOIN(
						SELECT
							pp.id, 
							SUM(sml.qty_done) AS qty_import
						FROM stock_move_line sml
							LEFT JOIN stock_location sl ON sml.location_dest_id=sl.id
							LEFT JOIN stock_picking sp ON sml.picking_id=sp.id
							LEFT JOIN product_product pp ON sml.product_id=pp.id
						WHERE sml.location_dest_id IN ('3835','1') -- địa điểm đích là kho tổng
							AND sml.location_id IN ('8','3823') -- địa điểm nguồn là vendor, kho xử lý
							AND sml.state = 'done'
                            AND sml.date + '7 hour' <= '2023-07-01'
						GROUP BY 
							pp.id) AS import ON import.id = pp.id
					LEFT JOIN(
						SELECT
				         pp.id,
				         SUM(sml.qty_done) AS qty_export
				      FROM stock_move_line sml
				         LEFT JOIN stock_location sl ON sml.location_dest_id=sl.id
				         LEFT JOIN product_product pp ON sml.product_id=pp.id
				         LEFT JOIN product_template pt ON pp.product_tmpl_id=pt.id
				      WHERE sml.location_dest_id IN ('3823','8') -- địa điểm đích là kho xử lý và vendor
				         AND sml.location_id  IN ('3835','1') -- địa điểm nguồn là Kho tổng
				         AND sml.state = 'done'
				         AND sml.date + '7 hour' <= '2023-07-01'
				      GROUP BY pp.id) AS export ON export.id = pp.id) AS import ON import.id=pp.id
    WHERE s.nbdb < '2023-07-01'
        AND DATE(po.date_order + '7 hour') <= s.nbdb + INTERVAL '1 month'
        AND pp.default_code IN %s
    GROUP BY pt.name,
        s.nbdb,
        import.qty_import
""", (default_code,)
)

data_old = old_cursor.fetchall()
columns_old = [name[0] for name in old_cursor.description]
df_old = pd.DataFrame(data_old,columns=columns_old)

In [359]:
cat_new_cursor = new_erp.cursor()
cat_new_cursor.execute(
    """
    SELECT
	DISTINCT(TRIM(SPLIT_PART(pc.complete_name,'/',1))) AS cat1,
	TRIM(SPLIT_PART(pc.complete_name,'/',2)) AS cat2,
	TRIM(SPLIT_PART(pc.complete_name,'/',3)) AS cat3,
	pc1.name AS collection,
	pp.default_code AS pfcode
FROM product_product pp
	LEFT JOIN product_template pt ON pp.product_tmpl_id=pt.id
	LEFT JOIN product_category pc ON pt.categ_id=pc.id
	LEFT JOIN product_collection pc1 ON pt.collection_id=pc1.id
WHERE pt.default_code in %s
	AND pp.active = TRUE """, (default_code,)
)
data_cat_new = cat_new_cursor.fetchall()
columns_cat = [name[0] for name in cat_new_cursor.description]
df_cat_new = pd.DataFrame(data_cat_new,columns=columns_cat)
df_old = pd.merge(df_old, df_cat_new, on = 'pfcode', how = 'left')
df_old

,default_code,color,size,nbdb,qty_import,qty_fm,total_amount_fm,cat1,cat2,cat3,collection,pfcode


# Query data Odoo mới

In [360]:
new_cursor = new_erp.cursor()
new_cursor.execute(
 """WITH import AS
	(SELECT 
		pp.default_code AS pfcode,
		SUM(sml.qty_done) - (CASE WHEN xuat.qty_export IS NULL THEN 0 ELSE xuat.qty_export END) AS qty_import,
		stock.qty_stock
	FROM stock_picking sp
		LEFT JOIN stock_move sm ON sp.id=sm.picking_id
		LEFT JOIN stock_picking_type spt ON sp.picking_type_id=spt.id
		LEFT JOIN stock_move_line sml ON sm.id=sml.move_id
		LEFT JOIN product_product pp ON sml.product_id=pp.id
		LEFT JOIN
			(SELECT 
				pp.default_code AS pfcode,
				SUM(sml.qty_done) AS qty_export
			FROM stock_picking sp
				LEFT JOIN stock_move sm ON sp.id=sm.picking_id
				LEFT JOIN stock_picking_type spt ON sp.picking_type_id=spt.id
				LEFT JOIN stock_move_line sml ON sm.id=sml.move_id
				LEFT JOIN product_product pp ON sml.product_id=pp.id
			WHERE sp.state = 'done'
				AND pp.active = TRUE
				AND sp.location_dest_id IN ('5597','8')
			GROUP BY pp.default_code) AS xuat ON xuat.pfcode = pp.default_code
		LEFT JOIN
			(SELECT
				pp.id,
				SUM(sq.quantity) AS qty_stock
			FROM stock_quant sq
				LEFT JOIN product_product pp ON sq.product_id=pp.id
				LEFT JOIN stock_location sl ON sq.location_id=sl.id
				LEFT JOIN stock_warehouse sw ON sl.warehouse_id=sw.id
			WHERE sl."usage" = 'internal'
				AND TRIM(SPLIT_PART(sw.name,'-',2)) NOT IN ('WEBLIME')
			GROUP BY pp.id) AS stock ON stock.id=pp.id
	WHERE spt.name IN ('Nhận hàng')
		AND sp.location_dest_id = 5565
		AND sp.location_id = 8
		AND sp.state = 'done'
		AND pp.active = TRUE
	GROUP BY	pp.default_code,
		stock.qty_stock,
		xuat.qty_export)

SELECT
	DISTINCT(TRIM(SPLIT_PART(pc.complete_name,'/',1))) AS cat1,
	TRIM(SPLIT_PART(pc.complete_name,'/',2)) AS cat2,
	TRIM(SPLIT_PART(pc.complete_name,'/',3)) AS cat3,
	pc1.name AS collection,
	pp.default_code AS pfcode,
	SPLIT_PART(pp.default_code,'-',1) AS default_code,
	SPLIT_PART(pp.default_code,'-',2) AS color,
	SPLIT_PART(pp.default_code,'-',3) AS size,
	fm.nbdb,
	import.qty_import,
	import.qty_stock,
	fm.qty AS qty_fm,
	fm.total_amount AS total_amount_fm,
	nm.qty_nm,
	nm.total_amount_nm
FROM pos_order po
	LEFT JOIN pos_order_line pol ON po.id=pol.order_id
	LEFT JOIN product_product pp ON pol.product_id=pp.id
	LEFT JOIN import ON import.pfcode = pp.default_code
	LEFT JOIN product_template pt ON pp.product_tmpl_id=pt.id
	LEFT JOIN product_category pc ON pt.categ_id=pc.id
	LEFT JOIN product_collection pc1 ON pt.collection_id=pc1.id
	RIGHT JOIN 
		(SELECT
			pol.product_id,
			s.nbdb,
			SUM(pol.qty) AS qty,
			SUM(pol.price_subtotal_incl) AS total_amount
		FROM pos_order po
			LEFT JOIN pos_order_line pol ON po.id=pol.order_id
			LEFT JOIN product_product pp ON pol.product_id=pp.id
			LEFT JOIN product_template pt ON pp.product_tmpl_id=pt.id
			LEFT JOIN 
				(SELECT
					pol.product_id,
					MIN(DATE(po.date_order + '7 hour')) AS nbdb
				FROM pos_order po
					LEFT JOIN pos_order_line pol ON po.id=pol.order_id
				GROUP BY pol.product_id) AS s ON s.product_id = pol.product_id
		WHERE DATE(po.date_order + '7 hour') <= s.nbdb + INTERVAL '1 month'
			AND pp.active = TRUE
		GROUP BY pol.product_id,
			s.nbdb) AS fm ON fm.product_id = pp.id
	LEFT JOIN
		(SELECT
			pol.product_id,
			SUM(pol.qty) AS qty_nm,
			SUM(pol.price_subtotal_incl) AS total_amount_nm
		FROM pos_order po	
			LEFT JOIN pos_order_line pol ON po.id=pol.order_id
		WHERE DATE(po.date_order + '7 hour') >= CURRENT_DATE - INTERVAL '1 month'
		GROUP BY pol.product_id) AS nm ON nm.product_id=pp.id
WHERE pt.default_code in %s""",(default_code,)
)
data_new = new_cursor.fetchall()
columns_new = [name[0] for name in new_cursor.description]
df_new = pd.DataFrame(data_new,columns=columns_new)
df_new

,cat1,cat2,cat3,collection,pfcode,default_code,color,size,nbdb,qty_import,qty_stock,qty_fm,total_amount_fm,qty_nm,total_amount_nm
0,FASHION TOP,ÁO THUN,ÁO THUN CROPTOP,Colorful Friends,LO23301110-LGR-L,LO23301110,LGR,L,2023-09-19,125,24.0,55,3948000.0,55,3948000.0
1,FASHION TOP,ÁO THUN,ÁO THUN CROPTOP,Colorful Friends,LO23301110-LGR-M,LO23301110,LGR,M,2023-09-19,143,21.0,64,2961000.0,64,2961000.0
2,FASHION TOP,ÁO THUN,ÁO THUN CROPTOP,Colorful Friends,LO23301110-LGR-S,LO23301110,LGR,S,2023-09-19,118,20.0,41,2961000.0,41,2961000.0


In [361]:
if df_old.empty:
    df_odoo = df_new
else:
    # Drop columns that doesn't correct, because all the SKUs happen to have the date begin sell out from 1/7/2023 (because I query the start date sale is the min date of pos order date so when transform to the new data, I need to drop the begin sell date of odoo new)
    df_new.drop(['nbdb', 'qty_fm', 'total_amount_fm'], axis=1, inplace=True)
    
    # Merge df_new with df_old
    df_odoo = pd.merge(df_new, df_old, on=['pfcode','default_code','color','size','cat1','cat2','cat3','collection'], how='outer')
    df_odoo.fillna(0,inplace=True)
    df_odoo['qty_import']=df_odoo['qty_import_x']+df_odoo['qty_import_y']
    df_odoo.drop(['qty_import_x','qty_import_y'],axis=1, inplace=True)
    df_odoo= df_odoo.reindex(['cat1','cat2','cat3','collection','pfcode','default_code','color','size','nbdb','qty_stock','qty_import','qty_fm','total_amount_fm','qty_nm','total_amount_nm'],axis=1)
df_odoo

,cat1,cat2,cat3,collection,pfcode,default_code,color,size,nbdb,qty_import,qty_stock,qty_fm,total_amount_fm,qty_nm,total_amount_nm
0,FASHION TOP,ÁO THUN,ÁO THUN CROPTOP,Colorful Friends,LO23301110-LGR-L,LO23301110,LGR,L,2023-09-19,125,24.0,55,3948000.0,55,3948000.0
1,FASHION TOP,ÁO THUN,ÁO THUN CROPTOP,Colorful Friends,LO23301110-LGR-M,LO23301110,LGR,M,2023-09-19,143,21.0,64,2961000.0,64,2961000.0
2,FASHION TOP,ÁO THUN,ÁO THUN CROPTOP,Colorful Friends,LO23301110-LGR-S,LO23301110,LGR,S,2023-09-19,118,20.0,41,2961000.0,41,2961000.0


In [362]:
# Sale information from online
sql_web = web.cursor()

# Generate the placeholders for the IN clause dynamically
placeholders = ', '.join(['%s'] * len(default_code))

sql_web.execute(
    """
    SELECT 
	od.PRD_CODE AS pfcode,
	s.nbdb, 
	SUM(od.ORD_QTY) AS qty_fm, 
	SUM(od.ORD_QTY * od.SALE_PRICE) AS total_amount_fm,
	qty_nm,
	total_amount_nm,
	pm.QTY_CUR AS qty_stock
FROM TBL_ORD_MAIN om
	LEFT JOIN TBL_ORD_DETAIL od ON om.ORD_CODE=od.ORD_CODE
	LEFT JOIN TBL_PRD_MAIN pm ON od.PRD_CODE=pm.PRD_CODE
	JOIN(
		SELECT
			od.PRD_CODE, 
			DATE(MIN(om.C_TIME)) AS nbdb
		FROM TBL_ORD_MAIN om
			LEFT JOIN TBL_ORD_DETAIL od ON om.ORD_CODE=od.ORD_CODE
		WHERE om.ORD_STATUS BETWEEN 60 AND 82 AND om.ORD_STATUS != 61
		GROUP BY od.PRD_CODE) AS s ON s.PRD_CODE=od.PRD_CODE
	LEFT JOIN(
		SELECT
			od.PRD_CODE AS pfcode, 
			SUM(od.ORD_QTY) AS qty_nm, 
			SUM(od.ORD_QTY * od.SALE_PRICE) AS total_amount_nm
		FROM TBL_ORD_MAIN om
			LEFT JOIN TBL_ORD_DETAIL od ON om.ORD_CODE=od.ORD_CODE
		WHERE om.ORD_STATUS BETWEEN 60 AND 82 
			AND om.ORD_STATUS != 61 
			AND DATE(om.C_TIME) >= CURRENT_DATE - INTERVAL 1 MONTH
		GROUP BY od.PRD_CODE) AS nm ON nm.pfcode=od.prd_code
WHERE om.ORD_STATUS BETWEEN 60 AND 82 
	AND om.ORD_STATUS != 61 
	AND DATE(om.C_TIME) <= s.nbdb + INTERVAL 1 MONTH 
	AND SUBSTRING_INDEX(od.PRD_CODE,'-',1) IN ({})
GROUP BY 
	s.nbdb,
	pm.QTY_CUR,
	od.PRD_CODE,
	qty_nm,
	total_amount_nm
    """.format(placeholders),
    default_code,  # Pass the values as a tuple
)

data_onl = sql_web.fetchall()
column_onl = [name[0] for name in sql_web.description]
df_onl = pd.DataFrame(data_onl, columns=column_onl)
df_onl.fillna(0,inplace=True)
df_onl

,pfcode,nbdb,qty_fm,total_amount_fm,qty_nm,total_amount_nm,qty_stock
0,LO23301110-LGR-S,2023-09-15,27,6463000,27,6463000,6
1,LO23301110-LGR-M,2023-09-15,23,4647000,23,4647000,8
2,LO23301110-LGR-L,2023-09-15,14,3246000,14,3246000,10


In [363]:
size_dict = {
    "S":'1.S',
    'M':'2.M',
    'L':'3.L',
    'XL':'4.XL',
    'XXL':'5.XXL',
    'Plus':'6.Plus',
    '27':'1.27',
    '28':'2.28',
    '29':'3.29',
    '30':'4.30',
    '31':'5.31',
    '32':'6.32'}

In [364]:
today = date.today()
if df_onl.empty:
    df_merge = df_odoo
    df_merge['pic']='=IMAGE(REPLACE("https://lo-product-images.s3-ap-southeast-1.amazonaws.com/thumbs/123.jpg",66,3,"'+df_merge['default_code']+'-'+df_merge['color']+'"))'
    df_merge['moh']= np.where(df_merge['qty_nm'] != 0, df_merge['qty_stock'].astype(float) / df_merge['qty_nm'].astype(float), 0)
    df_merge['nbdb'] = pd.to_datetime(df_merge['nbdb'])
    df_merge['month_sale'] = (today.year - df_merge['nbdb'].dt.year) * 12 + (today.month - df_merge['nbdb'].dt.month)

elif df_odoo.empty:
    df_merge = df_onl
    df_merge['pic']='=IMAGE(REPLACE("https://lo-product-images.s3-ap-southeast-1.amazonaws.com/thumbs/123.jpg",66,3,"'+df_merge['default_code']+'-'+df_merge['color']+'"))'
    df_merge['moh']= np.where(df_merge['qty_nm'] != 0, df_merge['qty_stock'].astype(float) / df_merge['qty_nm'].astype(float), 0)
    df_merge['nbdb'] = pd.to_datetime(df_merge['nbdb'])
    df_merge['month_sale'] = (today.year - df_merge['nbdb'].dt.year) * 12 + (today.month - df_merge['nbdb'].dt.month)
else:
    df_merge = pd.merge(df_odoo,df_onl,on='pfcode',how='outer')
    df_merge['nbdb'] = pd.to_datetime(df_merge[['nbdb_x','nbdb_y']].min(axis = 1))
    df_merge['qty_stock'] = pd.to_numeric(df_merge['qty_stock_x']+df_merge['qty_stock_y'])
    df_merge['qty_fm'] = df_merge['qty_fm_x']+df_merge['qty_fm_y']
    df_merge['total_amount_fm'] = df_merge['total_amount_fm_x']+df_merge['total_amount_fm_y']
    df_merge['qty_nm'] = pd.to_numeric(df_merge['qty_nm_x']+df_merge['qty_nm_y'])
    df_merge['total_amount_nm'] = df_merge['total_amount_nm_x']+df_merge['total_amount_nm_y']
    df_merge['pic']='=IMAGE(REPLACE("https://lo-product-images.s3-ap-southeast-1.amazonaws.com/thumbs/123.jpg",66,3,"'+df_merge['default_code']+'-'+df_merge['color']+'"))'
    df_merge['moh']= np.where(df_merge['qty_nm'] != 0, df_merge['qty_stock']/df_merge['qty_nm'], 0)
    df_merge['month_sale'] = (today.year - df_merge['nbdb'].dt.year) * 12 + (today.month - df_merge['nbdb'].dt.month)
    df_merge.drop(['nbdb_x','nbdb_y','qty_stock_x','qty_stock_y','total_amount_fm_x','total_amount_fm_y','qty_fm_x','qty_fm_y','total_amount_nm_x','total_amount_nm_y','qty_nm_x','qty_nm_y'],axis=1,inplace=True)
df_merge['size']=df_merge['size'].map(size_dict)
df_merge = df_merge.reindex(['cat1','cat2','cat3','collection','pfcode','default_code','color','size','nbdb','month_sale','pic','qty_fm','total_amount_fm','qty_nm','total_amount_nm','qty_import','qty_stock','moh'],axis=1)
df_merge.columns = ['CATEGORY 1', 'CATEGORY 2', 'CATEGORY 3', 'COLLECTION', 'MÃ ĐẦY ĐỦ', 'MÃ', 'MÀU', 'SIZE', 'NBDB', 'SỐ THÁNG BÁN', 'PIC', 'SLB THÁNG ĐẦU TIÊN', 'DT THÁNG ĐẦU TIÊN', 'SLB THÁNG GẦN ĐÂY', 'DT THÁNG GẦN ĐÂY', 'SL NHẬP','SL TỒN','MOH']
df_merge

,CATEGORY 1,CATEGORY 2,CATEGORY 3,COLLECTION,MÃ ĐẦY ĐỦ,MÃ,MÀU,SIZE,NBDB,SỐ THÁNG BÁN,PIC,SLB THÁNG ĐẦU TIÊN,DT THÁNG ĐẦU TIÊN,SLB THÁNG GẦN ĐÂY,DT THÁNG GẦN ĐÂY,SL NHẬP,SL TỒN,MOH
0,FASHION TOP,ÁO THUN,ÁO THUN CROPTOP,Colorful Friends,LO23301110-LGR-L,LO23301110,LGR,3.L,2023-09-15,0,"=IMAGE(REPLACE(""https://lo-product-images.s3-a...",69,7194000.0,69.0,7194000.0,125,34.0,0.492754
1,FASHION TOP,ÁO THUN,ÁO THUN CROPTOP,Colorful Friends,LO23301110-LGR-M,LO23301110,LGR,2.M,2023-09-15,0,"=IMAGE(REPLACE(""https://lo-product-images.s3-a...",87,7608000.0,87.0,7608000.0,143,29.0,0.333333
2,FASHION TOP,ÁO THUN,ÁO THUN CROPTOP,Colorful Friends,LO23301110-LGR-S,LO23301110,LGR,1.S,2023-09-15,0,"=IMAGE(REPLACE(""https://lo-product-images.s3-a...",68,9424000.0,68.0,9424000.0,118,26.0,0.382353


In [365]:
sheet1.batch_clear([f'A1:R'])
gd.set_with_dataframe(sheet1,df_merge)